### Imports

In [1]:
import os
import joblib
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler

import warnings
warnings.filterwarnings('ignore') 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # hopefully nothing explodes

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
print(tf.version)
print(tf.config.list_physical_devices('GPU'))

<module 'tensorflow._api.v2.version' from '/home/zyzz/anaconda3/lib/python3.11/site-packages/tensorflow/_api/v2/version/__init__.py'>
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# For reproducible results
seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)  

## Data

In [3]:
# Constants
val_size = 0.2
model_path = 'LSTM_v4'
data_path = 'training_dataset'
seq_length = 128     # predictions based on previous seq_length data entries
forecast_length = 9  # predicting forecast_length time steps into the future
sample_length = seq_length + forecast_length

In [4]:
# Read data (ignore categories)
training_data = np.load(os.path.join(data_path, 'training_data.npy'))
valid_periods = np.load(os.path.join(data_path, 'valid_periods.npy'))

# Filter out unvalid data
data = []
for i, row in enumerate(training_data):
    data.append(row[valid_periods[i][0]:valid_periods[i][1]])

print(f"({len(data)}, -)", valid_periods.shape)

(48000, -) (48000, 2)


In [5]:
# Convert time series to {x: sequences of length seq_length, y: values to be predicted from previous sequence}
def to_sequences(time_series):
    
    x = []
    y = []
    
    for i in range(time_series.shape[0]-seq_length-forecast_length+1):
        x.append(time_series[i:i+seq_length])
        y.append(time_series[i+seq_length:i+seq_length+forecast_length])  
    
    x = np.array(x)
    y = np.array(y)
    
    return {'x': x, 'y': y}

# Shuffle data (we don't want to make any assumptions about the order)
np.random.shuffle(data)
 
# Build sequences from the non-correlated time series, and append them to corresponding data set
# Note: there is no overlap between train and validation; each processed time series is used in train xor val
X_train, X_val = [], []
y_train, y_val = [], []
split_index = int((1-val_size)*len(data))
for i, time_series in enumerate(data): 
    if (len(time_series) >= sample_length): # assert we can draw at least one sample from the time_series
        sequences = to_sequences(time_series)
        if(i < split_index):
            X_train.append(sequences['x']) 
            y_train.append(sequences['y'])   
        else:
            X_val.append(sequences['x']) 
            y_val.append(sequences['y'])  

# Convert lists to nparrays 
X_train = np.concatenate(X_train, axis=0)
X_val = np.concatenate(X_val, axis=0)
y_train = np.concatenate(y_train, axis=0)
y_val = np.concatenate(y_val, axis=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(3373049, 128) (847359, 128) (3373049, 9) (847359, 9)


In [6]:
# Apply robust scaling (fit only to training data to avoid bias)
rscaler_X = RobustScaler().fit(X_train)
rscaler_y = RobustScaler().fit(y_train)
X_train = rscaler_X.transform(X_train)
X_val = rscaler_X.transform(X_val)
y_train = rscaler_y.transform(y_train)
y_val = rscaler_y.transform(y_val)

# Save scalers
joblib.dump(rscaler_X, os.path.join(model_path, 'rscaler_X.save'))
joblib.dump(rscaler_y, os.path.join(model_path, 'rscaler_y.save'))

# Add the time dimension to the data sets
X_train = X_train.reshape((-1, seq_length, 1))
X_val = X_val.reshape((-1, seq_length, 1))                      
y_train = y_train.reshape((-1, forecast_length, 1)) 
y_val = y_val.reshape((-1, forecast_length, 1))   
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)                      

(3373049, 128, 1) (847359, 128, 1) (3373049, 9, 1) (847359, 9, 1)


## ML

In [7]:
input_shape = (X_train.shape[1], X_train.shape[2])
dropout_rate = 0.2
batch_size = 128
lstm_units = 128
epochs = 1000

def build_model(input_shape, lstm_units, dropout_rate):
    input_layer = tfkl.Input(shape=input_shape)
      
    # Block one; bidirectional LSTMs with regularizers, BNormalization and dropout
    x = tfkl.Bidirectional(
        tfkl.LSTM(units=lstm_units, return_sequences=True, kernel_regularizer=tfk.regularizers.l2(0.001)))(input_layer)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(dropout_rate)(x)

    # Block two; similar to previous but no bidirectionality or regularizer
    x = tfkl.LSTM(units=lstm_units // 2)(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(dropout_rate)(x)
    x = tfkl.Dense(units=forecast_length)(x)

    # Compile using MSE as loss function and the Adam optimizer
    model = tfk.Model(inputs=input_layer, outputs=x)
    model.compile(optimizer='adam', loss='mse')

    return model

In [8]:
# Stop training when validation loss stops improving, maintain best weights
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,         # how many epochs to check for improvement before stopping
    restore_best_weights=True,
)
    
# Build and train model
model = build_model(input_shape, lstm_units, dropout_rate)
history = model.fit(X_train,
                    y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_data=(X_val, y_val),
                    callbacks=early_stopping,
                    verbose=1)

Epoch 1/1000


I0000 00:00:1702974428.718579     570 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


26352/26352 [==============================] - 666s 25ms/step - loss: 0.0537 - val_loss: 0.0374
Epoch 2/1000
26352/26352 [==============================] - 626s 24ms/step - loss: 0.0432 - val_loss: 0.0361
Epoch 3/1000
26352/26352 [==============================] - 619s 24ms/step - loss: 0.0420 - val_loss: 0.0364
Epoch 4/1000
26352/26352 [==============================] - 616s 23ms/step - loss: 0.0414 - val_loss: 0.0365
Epoch 5/1000
26352/26352 [==============================] - 611s 23ms/step - loss: 0.0442 - val_loss: 0.0383
Epoch 6/1000
26352/26352 [==============================] - 614s 23ms/step - loss: 0.0448 - val_loss: 0.0367
Epoch 7/1000
26352/26352 [==============================] - 607s 23ms/step - loss: 0.0420 - val_loss: 0.0431
Epoch 8/1000
26352/26352 [==============================] - 608s 23ms/step - loss: 0.0409 - val_loss: 0.0354
Epoch 9/1000
26352/26352 [==============================] - 609s 23ms/step - loss: 0.0405 - val_loss: 0.0337
Epoch 10/1000
26352/26352 [=====

In [9]:
# Evaluate on original validation data 
y_val_org = rscaler_y.inverse_transform(y_val.reshape((-1, forecast_length)))
y_pred = model.predict(X_val)
y_pred_iscaled = rscaler_y.inverse_transform(y_pred.reshape((-1, forecast_length)))
mse = tfk.losses.MeanSquaredError()
print(f"Val loss (MSE): {mse(y_val_org, y_pred_iscaled).numpy()}")

# Val loss for each prediction step
for t in range(forecast_length):
    mse = tfk.metrics.MeanSquaredError()
    mse.update_state(y_val_org[:, t], y_pred_iscaled[:, t])
    print(f'Val loss (MSE) {t+1} step forward: {mse.result().numpy()}')

26480/26480 [==============================] - 185s 7ms/step
Val loss (MSE): 0.005323225166648626
Val loss (MSE) 1 step forward: 0.002490357495844364
Val loss (MSE) 2 step forward: 0.003336182562634349
Val loss (MSE) 3 step forward: 0.004094033967703581
Val loss (MSE) 4 step forward: 0.00481325201690197
Val loss (MSE) 5 step forward: 0.005442761350423098
Val loss (MSE) 6 step forward: 0.006037930026650429
Val loss (MSE) 7 step forward: 0.006666239351034164
Val loss (MSE) 8 step forward: 0.007232982665300369
Val loss (MSE) 9 step forward: 0.007795285899192095


In [10]:
# save model
model.save(os.path.join(model_path, 'model'))

INFO:tensorflow:Assets written to: LSTM_v4/model/assets


INFO:tensorflow:Assets written to: LSTM_v4/model/assets
